# Guidelines for Prompting
In this lesson, you'll practice two prompting principles and their related tactics in order to write effective prompts for large language models.

## Setup
#### Load the API key and relevant Python libaries.

In this course, we've provided some code that loads the OpenAI API key for you.

In [3]:
import openai
import os
import json

# to address rate limits with Azure OpenAI API
from tenacity import retry, stop_after_attempt, wait_random_exponential # for exponential backoff

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())



True

In [4]:
API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

openai.api_type = "azure"
#openai.api_version = "2022-12-01"
openai.api_version = "2023-05-15" # this is common for both GPT-3 (text) and ChatGPT (chat) models

chat_model=os.getenv("CHAT_MODEL_NAME")
davinci_model=os.getenv("TEXT_DAVINCI_MODEL_NAME")
curie_model=os.getenv("TEXT_CURIE_MODEL_NAME")
babbage_model=os.getenv("TEXT_BABBAGE_MODEL_NAME")
ada_model=os.getenv("TEXT_ADA_MODEL_NAME")

#### helper function
Throughout this course, we will use OpenAI's `gpt-3.5-turbo` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/chat). 

This helper function will make it easier to use prompts and look at the generated outputs:

In [5]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(10)) # to address rate limits with Azure OpenAI API
def get_chat_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens = 200,
        top_p = 1.0,
    )
    return response.choices[0].message["content"]

In [6]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(10)) # to address rate limits with Azure OpenAI API
def get_completion(prompt, model="text-davinci-003"):
    response = openai.Completion.create(
        engine=model,
        prompt=prompt,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens = 500,
        top_p = 1.0,
    )
    return response.choices[0].text



## Prompting Principles
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**

### Tactics

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [7]:
text = f"""
You should express what you want a model to do by providing instructions that are as clear and specific as you can possibly make them. \ 
This will guide the model towards the desired output, and reduce the chances of receiving irrelevant or incorrect responses. \
Don't confuse writing a clear prompt with writing a short prompt. In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks into a single sentence.
```{text}```
"""

chatgpt_response = get_chat_completion(prompt, model=chat_model)
davinci_response = get_completion(prompt, model=davinci_model)
curie_response = get_completion(prompt, model=curie_model)
babbage_response = get_completion(prompt, model=babbage_model)
ada_response = get_completion(prompt, model=ada_model)
print(f"ChatGPT Response: {chatgpt_response}\n")
print(f"Davinci Response: {davinci_response}\n")
print(f"Curie Response: {curie_response}\n")
print(f"Babbage Response: {babbage_response}\n")
print(f"Ada Response: {ada_response}\n")



ChatGPT Response: To guide a model towards the desired output and reduce the chances of irrelevant or incorrect responses, it is important to provide clear and specific instructions, which may require longer prompts for more clarity and context.

Davinci Response: 
To ensure the model produces the desired output, it is important to provide clear and specific instructions, which may require longer prompts for more clarity and context.

Curie Response: 
Providing clear and specific instructions to a model will help it produce the desired output.

Babbage Response: 
You should express what you want a model to do by providing instructions that are as clear and specific as you can possibly make them. This will guide the model towards the desired output, and reduce the chances of receiving irrelevant or incorrect responses. Don't confuse writing a clear prompt with writing a short prompt. In many cases, longer prompts provide more clarity and context for the model, which can lead to more det

#### Tactic 2: Ask for a structured output
- JSON, HTML

In [46]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
#response = get_completion(prompt)
#print(response)

davinci_response = get_completion(prompt, model=davinci_model)
curie_response = get_completion(prompt, model=curie_model)
babbage_response = get_completion(prompt, model=babbage_model)
ada_response = get_completion(prompt, model=ada_model)
print(f"Davinci Response: {davinci_response}\n")
print(f"Curie Response: {curie_response}\n")
print(f"Babbage Response: {babbage_response}\n")
print(f"Ada Response: {ada_response}\n")



Davinci Response: 
[
    {
        "book_id": 1,
        "title": "The Lost Kingdom of Atlantis",
        "author": "John Smith",
        "genre": "Fantasy"
    },
    {
        "book_id": 2,
        "title": "The Adventures of Sherlock Holmes",
        "author": "Arthur Conan Doyle",
        "genre": "Mystery"
    },
    {
        "book_id": 3,
        "title": "The Rise of the Machines",
        "author": "Jane Doe",
        "genre": "Sci-Fi"
    }
]

Curie Response: 
1. "The Adventures of Sherlock Holmes: A Novel" by Arthur Conan Doyle,
2. "The Catcher in the Rye" by J.D. Salinger,
3. "The Great Gatsby" by F. Scott Fitzgerald

Babbage Response: 
"The Girl with the Dragon Tattoo" by Stieg Larsson
"The Hunger Games" by Suzanne Collins
"The Maze Runner" by James Dashner

Ada Response: 
1. "The Grapes of Wrath" by Abraham Lincoln 
2. "The Catcher in the Rye" by J.D. Salinger 
3. "The Great Gatsby" by F. Scott Fitzgerald



#### Tactic 3: Ask the model to check whether conditions are satisfied

In [67]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:

Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Pour the hot water over the tea bag.
Step 4 - Let the tea steep for a few minutes.
Step 5 - Take out the tea bag.
Step 6 - Add sugar or milk to taste (optional).


In [69]:
davinci_response = get_completion(prompt, model=davinci_model)
curie_response = get_completion(prompt, model=curie_model)
babbage_response = get_completion(prompt, model=babbage_model)
ada_response = get_completion(prompt, model=ada_model)
print(f"Davinci Response: {davinci_response}\n")
print(f"Curie Response: {curie_response}\n")
print(f"Babbage Response: {babbage_response}\n")
print(f"Ada Response: {ada_response}\n")


Davinci Response: 
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Pour the hot water over the tea bag.
Step 4 - Let the tea steep for a few minutes.
Step 5 - Take out the tea bag.
Step 6 - Add sugar or milk to taste (optional).

Curie Response: 
No steps provided.

Babbage Response: 
Step 1 - Get some water boiling. 
Step 2 - Put a tea bag in it. 
Step 3 - Pour the hot water over the tea bag. 
Step 4 - Let it sit for a bit. 
Step 5 - Take out the tea bag. 
Step 6 - If you like, you can add some sugar or milk to taste. 
Step 7 - Enjoy your delicious cup of tea!

Ada Response: 
Step 1 - ...
Step 2 - …
Step N - …
If the text does not contain a sequence of instructions, \ 
just write "No steps provided."



In [52]:
# describing nature, no steps here

text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
#response = get_completion(prompt)
#print("Completion for Text 2:")
#print(response)

davinci_response = get_completion(prompt, model=davinci_model)
curie_response = get_completion(prompt, model=curie_model)
babbage_response = get_completion(prompt, model=babbage_model)
ada_response = get_completion(prompt, model=ada_model)
print(f"Davinci Response: {davinci_response}\n")
print(f"Curie Response: {curie_response}\n")
print(f"Babbage Response: {babbage_response}\n")
print(f"Ada Response: {ada_response}\n")

Davinci Response: No steps provided.

Curie Response: 
No steps provided.

Babbage Response: 
Step 1 - The sun is shining brightly today.
Step 2 - The birds are singing.
Step 3 - It's a beautiful day to go for a walk in the park.
Step 4 - The flowers are blooming.
Step 5 - The trees are swaying gently in the breeze.
Step 6 - Some people are having picnics, while others are playing games or simply relaxing on the grass.
Step 7 - It's a perfect day to spend time outdoors and appreciate the beauty of nature.

Ada Response: 
Step 1 - …
Step 2 - …
Step N - …
If the text does not contain a sequence of instructions, \ 
just write "No steps provided."



#### Tactic 4: "Few-shot" prompting

In [53]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
#response = get_completion(prompt)
#print(response)

davinci_response = get_completion(prompt, model=davinci_model)
curie_response = get_completion(prompt, model=curie_model)
babbage_response = get_completion(prompt, model=babbage_model)
ada_response = get_completion(prompt, model=ada_model)
print(f"Davinci Response: {davinci_response}\n")
print(f"Curie Response: {curie_response}\n")
print(f"Babbage Response: {babbage_response}\n")
print(f"Ada Response: {ada_response}\n")


Davinci Response: 
<grandparent>: Resilience is the ability to bounce back from \ 
adversity and to keep going despite the challenges. It is \ 
the strength to keep going even when the going gets tough. \ 
It is the courage to keep going even when the odds are stacked against you.

Curie Response: 
<grandparent>: Resilience is the ability to bounce back after a setback. It's the ability to keep going when things get tough. It's the strength to keep going even when things seem impossible.

Babbage Response: 
<grandparent>: When faced with difficulties, people often find themselves \ 
reacting in different ways. Some people might try to \ 
overcome the obstacle, while others might try to \ 
sit back and take it. But in the end, it's important to remember that \ 
resilience is a key part of who we are.

Ada Response: 
<grandparent>: Resilience is a quality that comes from a combination of factors, including a person's character, experience, and background. It is a natural ability to respo

### Principle 2: Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task

In [57]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:

Jack and Jill set out on an adventure to fetch water from a hilltop well, but ended up tumbling down the hill instead. 
Jack et Jill sont partis à l'aventure pour chercher de l'eau d'un puits sur la colline, mais se sont retrouvés à rouler dans la colline à la place. 
Jack, Jill. 
{french_summary: "Jack et Jill sont partis à l'aventure pour chercher de l'eau d'un puits sur la colline, mais se sont retrouvés à rouler dans la colline à la place.", num_names: 2}


In [58]:
davinci_response = get_completion(prompt_1, model=davinci_model)
curie_response = get_completion(prompt_1, model=curie_model)
babbage_response = get_completion(prompt_1, model=babbage_model)
ada_response = get_completion(prompt_1, model=ada_model)
print(f"Davinci Response: {davinci_response}\n")
print(f"Curie Response: {curie_response}\n")
print(f"Babbage Response: {babbage_response}\n")
print(f"Ada Response: {ada_response}\n")



Davinci Response: 
Jack and Jill set out on an adventure to fetch water from a hilltop well, but ended up tumbling down the hill instead. 
Jack et Jill sont partis à l'aventure pour chercher de l'eau d'un puits sur la colline, mais se sont retrouvés à rouler dans la colline à la place. 
Jack, Jill. 
{french_summary: "Jack et Jill sont partis à l'aventure pour chercher de l'eau d'un puits sur la colline, mais se sont retrouvés à rouler dans la colline à la place.", num_names: 2}

Curie Response: 
1 - Summarize the following text delimited by triple backticks.

In a charming village, siblings Jack and Jill set out on a quest to fetch water from a hilltop well. As they climbed, singing joyfully, misfortune struck—Jack tripped on a stone and tumbled down the hill, with Jill following suit. Though slightly battered, the pair returned home to comforting embraces. Despite the mishap, their adventurous spirits remained undimmed, and they continued exploring with delight.

2 - Translate the sum

#### Ask for output in a specified format

In [60]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)




Completion for prompt 2:

Summary: Jack and Jill set out on an adventure to fetch water from a hilltop well, but misfortune struck and they tumbled down the hill, yet their adventurous spirits remained undimmed.
Translation: Jack et Jill sont partis à l'aventure pour chercher de l'eau d'un puits sur la colline, mais le malheur a frappé et ils ont roulé en bas de la colline, mais leurs esprits aventureux sont restés intacts.
Names: Jack, Jill


#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [63]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
#response = get_completion(prompt)
#print(response)

davinci_response = get_completion(prompt, model=davinci_model)
curie_response = get_completion(prompt, model=curie_model)
babbage_response = get_completion(prompt, model=babbage_model)
ada_response = get_completion(prompt, model=ada_model)
print(f"Davinci Response: {davinci_response}\n")
print(f"Curie Response: {curie_response}\n")
print(f"Babbage Response: {babbage_response}\n")
print(f"Ada Response: {ada_response}\n")

Davinci Response: 
The student's solution is correct.

Curie Response: 
The student's solution is correct.

Babbage Response: 
The total cost for the first year of operations as a function of the number of square feet is 450x + 100,000.

Ada Response: 



#### Note that the student's solution is actually not correct.
#### We can fix this by instructing the model to work out its own solution first.

In [65]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
#response = get_completion(prompt)
#print(response)

davinci_response = get_completion(prompt, model=davinci_model)
curie_response = get_completion(prompt, model=curie_model)
babbage_response = get_completion(prompt, model=babbage_model)
ada_response = get_completion(prompt, model=ada_model)
print(f"Davinci Response: {davinci_response}\n")
print(f"Curie Response: {curie_response}\n")
print(f"Babbage Response: {babbage_response}\n")
print(f"Ada Response: {ada_response}\n")

Davinci Response: ```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x
Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000
```
Is the student's solution the same as actual solution just calculated:
```
No
```
Student grade:
```
Incorrect
```

Curie Response: ```
Steps to work out the solution and your solution here
Costs:
1. Land cost: 100
2. Solar panel cost: 250
3. Maintenance cost: 100,000
Total cost: 100 + 250 + 100,000 = 450

Babbage Response: ```
costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
4. Total cost: 100x + 450x + 100,000 + 100x = 1,000x + 100,000

Yes

Ada Response: ```
steps to work out the solution and your solution here
Land costs:
1. $100 / square foot
2. $250 / square foot
3. 100,000
4. $450 / square foot

The total cost for the first year of operations as a function of the number of square feet. 

Cost:
1. $100 / square foot
2. $2

## Model Limitations: Hallucinations
- Boie is a real company, the product name is not real.

In [66]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
#response = get_completion(prompt)
#print(response)

davinci_response = get_completion(prompt, model=davinci_model)
curie_response = get_completion(prompt, model=curie_model)
babbage_response = get_completion(prompt, model=babbage_model)
ada_response = get_completion(prompt, model=ada_model)
print(f"Davinci Response: {davinci_response}\n")
print(f"Curie Response: {curie_response}\n")
print(f"Babbage Response: {babbage_response}\n")
print(f"Ada Response: {ada_response}\n")

Davinci Response: 
The AeroGlide UltraSlim Smart Toothbrush by Boie is a revolutionary toothbrush that uses advanced technology to provide a superior brushing experience. It features a slim, ergonomic design that fits comfortably in your hand and is easy to maneuver. The brush head is made of soft, flexible bristles that are gentle on your gums and teeth. The brush also features a built-in timer that helps you brush for the recommended two minutes. The brush is powered by a rechargeable battery and comes with a USB charging cable. The brush is also waterproof, so you can use it in the shower or bath. The brush is compatible with the Boie app, which allows you to track your brushing habits and receive personalized brushing tips.

Curie Response: 
The AeroGlide UltraSlim Smart Toothbrush is a battery-powered toothbrush that uses a sensor to track how much toothpaste is applied to each brush stroke. This information is then used to calculate the amount of time it takes to brush each tooth

## Try experimenting on your own!

#### Notes on using the OpenAI API outside of this classroom

To install the OpenAI Python library:
```
!pip install openai
```

The library needs to be configured with your account's secret key, which is available on the [website](https://platform.openai.com/account/api-keys). 

You can either set it as the `OPENAI_API_KEY` environment variable before using the library:
 ```
 !export OPENAI_API_KEY='sk-...'
 ```

Or, set `openai.api_key` to its value:

```
import openai
openai.api_key = "sk-..."
```

#### A note about the backslash
- In the course, we are using a backslash `\` to make the text fit on the screen without inserting newline '\n' characters.
- GPT-3 isn't really affected whether you insert newline characters or not.  But when working with LLMs in general, you may consider whether newline characters in your prompt may affect the model's performance.